In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

from share import *

from cldm.model import create_model, load_state_dict


# Configs
# resume_path = './models/sd21-unclip-h.ckpt'
# config_path='./models/v2-1-stable-unclip-h-inference.yaml'

# resume_path = './train_log/kin_hed_unclip4/lightning_logs/version_0/checkpoints/epoch=1-step=157130.ckpt'
resume_path = './train_log/kin_hed_unclip3/lightning_logs/version_4/checkpoints/epoch=1-step=159230.ckpt'

config_path='./models/cldm_unclip-h-inference.yaml'

# !!!!!!!!!!!!!!!!!!!!!!!!!!!\
# check where images config is set correct
# either 64 for 512 imgs or 96 for 768 imgs

# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model(config_path).cuda()

state_dict = load_state_dict(resume_path, location='cuda')
# state_dict.pop('model_ema.decay')
# state_dict.pop('model_ema.num_updates')

model.load_state_dict(state_dict)

# 'model_ema.decay', 'model_ema.num_updates'

logging improved.


/export/home/nstracke/miniconda3/envs/control/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/export/home/nstracke/miniconda3/envs/control/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
/export/home/nstracke/miniconda3/envs/control/lib/python3.8/site-packages/torch/distributed/_sharded_tensor/__init__.py:8: DeprecationWarning: torch.distributed._sharded_tensor will be deprecated, use torch.distributed._shard.sharded_tensor instead
  warnings.warn(


ControlLDM: Running in v-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 20 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, c

odict_keys(['betas', 'alphas_cumprod', 'alphas_cumprod_prev', 'sqrt_alphas_cumprod', 'sqrt_one_minus_alphas_cumprod', 'log_one_minus_alphas_cumprod', 'sqrt_recip_alphas_cumprod', 'sqrt_recipm1_alphas_cumprod', 'posterior_variance', 'posterior_log_variance_clipped', 'posterior_mean_coef1', 'posterior_mean_coef2', 'model.diffusion_model.time_embed.0.weight', 'model.diffusion_model.time_embed.0.bias', 'model.diffusion_model.time_embed.2.weight', 'model.diffusion_model.time_embed.2.bias', 'model.diffusion_model.label_emb.0.0.weight', 'model.diffusion_model.label_emb.0.0.bias', 'model.diffusion_model.label_emb.0.2.weight', 'model.diffusion_model.label_emb.0.2.bias', 'model.diffusion_model.input_blocks.0.0.weight', 'model.diffusion_model.input_blocks.0.0.bias', 'model.diffusion_model.input_blocks.1.0.in_layers.0.weight', 'model.diffusion_model.input_blocks.1.0.in_layers.0.bias', 'model.diffusion_model.input_blocks.1.0.in_layers.2.weight', 'model.diffusion_model.input_blocks.1.0.in_layers.2.b

In [2]:
import importlib
import torch
import cv2
import numpy as np
import PIL
from omegaconf import OmegaConf
from PIL import Image
from tqdm import trange
import io, os
from torch import autocast
from einops import rearrange, repeat
from torchvision.utils import make_grid
from pytorch_lightning import seed_everything
from contextlib import nullcontext

from ldm.models.diffusion.ddim import DDIMSampler
# from cldm.ddim_hacked import DDIMSampler

from ldm.models.diffusion.plms import PLMSSampler
from ldm.models.diffusion.dpm_solver import DPMSolverSampler

torch.set_grad_enabled(False)

In [3]:
# returns img in [0, 255]
def load_img(path):
    image = Image.open(path)
    w, h = image.size
    print(f"loaded input image of size ({w}, {h})")
    w, h = map(lambda x: x - x % 64, (w, h))
    image = image.resize((w, h), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32)
    image = image.transpose(2, 0, 1)
    return image

def get_img(path, batch_size=1):
    init_image = load_img(path)#.cuda()
    # init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
    return init_image


def sample(
        model,
        prompt,
        n_runs=3,
        n_samples=2,
        H=512,
        W=512,
        C=4,
        f=8,
        scale=10.0,
        ddim_steps=50,
        ddim_eta=0.0,
        callback=None,
        skip_single_save=False,
        save_grid=True,
        ucg_schedule=None,
        negative_prompt="",
        adm_cond=None,
        adm_uc=None,
        use_full_precision=False,
        only_adm_cond=False,
        control=None
):
    batch_size = n_samples
    precision_scope = autocast if not use_full_precision else nullcontext
    # decoderscope = autocast if not use_full_precision else nullcontext
    if isinstance(prompt, str):
        prompt = [prompt]
    prompts = batch_size * prompt
    
    with precision_scope("cuda"):
        with model.ema_scope():
            all_samples = list()
            for n in trange(n_runs, desc="Sampling"):
                shape = [C, H // f, W // f]
                if not only_adm_cond:
                    uc = None
                    if scale != 1.0:
                        uc = model.get_learned_conditioning(batch_size * [negative_prompt])
                    if isinstance(prompts, tuple):
                        prompts = list(prompts)
                    c = model.get_learned_conditioning(prompts)

                if adm_cond is not None:
                    if adm_cond.shape[0] == 1:
                        adm_cond = repeat(adm_cond, '1 ... -> b ...', b=batch_size)
                    if adm_uc is None:
                        st.warning("Not guiding via c_adm")
                        adm_uc = adm_cond
                    else:
                        if adm_uc.shape[0] == 1:
                            adm_uc = repeat(adm_uc, '1 ... -> b ...', b=batch_size)
                    if not only_adm_cond:
                        c = {"c_crossattn": [c], "c_adm": adm_cond, "c_concat": [control]}
                        uc = {"c_crossattn": [uc], "c_adm": adm_uc, "c_concat": [control]}
                    else:
                        c = adm_cond
                        uc = adm_uc
                samples_ddim, _ = sampler.sample(S=ddim_steps,
                                                 conditioning=c,
                                                 batch_size=batch_size,
                                                 shape=shape,
                                                 verbose=False,
                                                 unconditional_guidance_scale=scale,
                                                 unconditional_conditioning=uc,
                                                 eta=ddim_eta,
                                                 x_T=None,
                                                 callback=callback,
                                                 ucg_schedule=ucg_schedule
                                                 )
                x_samples = model.decode_first_stage(samples_ddim)
                x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)

                if not skip_single_save:
                    base_count = len(os.listdir(os.path.join(SAVE_PATH, "samples")))
                    for x_sample in x_samples:
                        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                        Image.fromarray(x_sample.astype(np.uint8)).save(
                            os.path.join(SAVE_PATH, "samples", f"{base_count:09}.png"))
                        base_count += 1

                all_samples.append(x_samples)

                # get grid of all samples
                grid = torch.stack(all_samples, 0)
                grid = rearrange(grid, 'n b c h w -> (n h) (b w) c')

            # additionally, save grid
            grid = Image.fromarray((255. * grid.cpu().numpy()).astype(np.uint8))
            if save_grid:
                grid_count = len(os.listdir(SAVE_PATH)) - 1
                grid.save(os.path.join(SAVE_PATH, f'grid-{grid_count:06}.png'))

    return x_samples



In [4]:
from dataset import MyDataset
from torch.utils.data import DataLoader
import glob

dataset = MyDataset('kin_hed2')
dataloader = DataLoader(dataset, num_workers=1, batch_size=1, shuffle=True)

def get_key(name: str):
    name = name.split('/')[-1] 
    name = name.split('.')[0] 
    name = name.split('_')[-1]
    name = name[3:]
    return int(name)

def get_sequence():
    for batch in dataloader:
        meta_batch = batch['meta']['file_name']
        meta = meta_batch[0]
        meta = meta.split('/')[1]
        meta = meta.split('_')[:-1]
        meta = '_'.join(meta)
        print(meta)
        styles = glob.glob('../ControlNet/data/kin_hed2/jpg/' + meta + '*')
        styles.sort(key=get_key)
        structures = glob.glob('../ControlNet/data/kin_hed2/hint/' + meta + '*')
        structures.sort(key=get_key)
        return styles, structures

In [5]:
def make_conditionings_from_input(img, num=1):
    with torch.no_grad():
        adm_cond = model.embedder(img)
        weight = 1
        if model.noise_augmentor is not None:
            noise_level = 0
            c_adm, noise_level_emb = model.noise_augmentor(adm_cond, noise_level=repeat(
                torch.tensor([noise_level]).to(model.device), '1 -> b', b=num))
            adm_cond = torch.cat((c_adm, noise_level_emb), 1) * weight
        adm_uc = torch.zeros_like(adm_cond)
    return adm_cond, adm_uc, weight

In [8]:
import torchvision

number_rows = 1
steps = 50
eta = 0
negative_prompt = ''
prompt = ''
scale = 10
SAVE_PATH = os.path.join('./video_check_first')

os.makedirs(os.path.join(SAVE_PATH, "samples"), exist_ok=True)
sampler = DDIMSampler(model)

for i in range(10):

    styles, structures = get_sequence()
    styles = np.array([get_img(style) for style in styles])
    styles = torch.from_numpy(styles) / 127.5 - 1.0
    styles = styles.cuda()

    structures = np.array([get_img(structure) for structure in structures])
    structures = torch.from_numpy(structures) / 255.0
    structures = structures.cuda()

    number_cols = len(structures)

    adm_inputs = list()
    weights = list()

    adm_cond, adm_uc, w = make_conditionings_from_input(styles[:1], num=len(structures))
    weights.append(w)
    adm_inputs.append(adm_cond)
    adm_cond = torch.stack(adm_inputs).sum(0) / sum(weights)

    print(adm_cond.shape)


    samples = sample(
                    model,
                    prompt,
                    n_runs=number_rows,
                    n_samples=number_cols,
                    H=512, W=512, C=4, f=8,
                    scale=scale,
                    ddim_steps=steps,
                    ddim_eta=eta,
                    ucg_schedule=None,
                    negative_prompt=negative_prompt,
                    adm_cond=adm_cond, adm_uc=adm_uc,
                    use_full_precision=False,
                    only_adm_cond=False,
                    save_grid=False,
                    skip_single_save=True,
                    control=structures
                )

    print(samples.shape)

    x_styles = styles * 127.5 + 127.5

    structures = structures * 255
    structures = torch.cat([torch.zeros_like(x_styles[:1]), structures])

    x_samples = samples * 255
    x_samples = torch.cat([torch.zeros_like(x_styles[:1]), x_samples])

    x_samples = torch.cat([x_styles, structures, x_samples], 2)
    x_samples = rearrange(x_samples, 'b c h w -> h (b w) c')
        
    grid = x_samples.cpu().numpy().clip(0, 255).astype(np.uint8)

    Image.fromarray(grid).save(os.path.join(SAVE_PATH, f'${i}.png'))


iueN8knNo8M_000000_000010
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
torch.Size([4, 2048])


Sampling:   0%|          | 0/1 [00:00<?, ?it/s]

Data shape for DDIM sampling is (4, 4, 64, 64), eta 0
Running DDIM Sampling with 50 timesteps


Sampling: 100%|██████████| 1/1 [00:06<00:00,  6.85s/it]


torch.Size([4, 3, 512, 512])
A71X4X7EV2A_000037_000047
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
torch.Size([4, 2048])


...

torch.Size([4, 3, 512, 512])
CggbH4pY6TA_000294_000304
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
loaded input image of size (512, 512)
torch.Size([4, 2048])


Sampling:   0%|          | 0/1 [00:00<?, ?it/s]

Data shape for DDIM sampling is (4, 4, 64, 64), eta 0
Running DDIM Sampling with 50 timesteps


Sampling: 100%|██████████| 1/1 [00:06<00:00,  6.83s/it]


torch.Size([4, 3, 512, 512])


torch.Size([5, 3, 512, 512])
torch.Size([1536, 2560, 3])
(1536, 2560, 3)
